In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Read the image
image_path = #enter your path here
image = cv2.imread(image_path)

# Check if the image was loaded correctly
if image is None:
    raise FileNotFoundError(f"The image at the path {image_path} could not be found. Please check the path.")

# Extract the red channel from the image
red_channel = image[:, :, 2]

# Create a binary mask where the red circles are marked as white areas and the background as black areas
_, binary_mask = cv2.threshold(red_channel, 50, 255, cv2.THRESH_BINARY)

# Apply morphological operations to close small gaps and connect the red areas
kernel = np.ones((21, 26), np.uint8)
binary_mask_dilated = cv2.dilate(binary_mask, kernel, iterations=2)
binary_mask_closed = cv2.erode(binary_mask_dilated, kernel, iterations=2)

# Find contours
contours, _ = cv2.findContours(binary_mask_closed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Find the largest contour (the largest connected red area)
largest_contour = max(contours, key=cv2.contourArea)

# Draw the contour of the largest connected area on the original image (for verification)
contour_image = image.copy()
cv2.drawContours(contour_image, [largest_contour], -1, (0, 255, 0), thickness=2)

# Create a new mask where only the largest connected red area remains
final_mask = np.zeros_like(binary_mask)
cv2.drawContours(final_mask, [largest_contour], -1, (255), thickness=cv2.FILLED)

# Additional masking to remove the lower black areas
height, width = final_mask.shape
# Color rectangular areas bottom left, bottom right, and bottom center in red
cv2.rectangle(final_mask, (0, int(0.9*height)), (int(0.1*width), height), (255), thickness=cv2.FILLED)
cv2.rectangle(final_mask, (int(0.9*width), int(0.9*height)), (width, height), (255), thickness=cv2.FILLED)
cv2.rectangle(final_mask, (int(0.45*width), int(0.9*height)), (int(0.55*width), height), (255), thickness=cv2.FILLED)

# Create the result image: Red for the largest connected area, black for the rest
result = np.zeros((height, width, 4), dtype=np.uint8)  # RGBA format

# Set the red channel and the alpha channel
result[:, :, 0] = 0           # R channel
result[:, :, 1] = 0           # G channel
result[:, :, 2] = final_mask  # R channel
result[:, :, 3] = final_mask  # A channel

# Make black areas (background) transparent
transparent_mask = final_mask == 0
result[transparent_mask] = [0, 0, 0, 0]

# Save the result image
output_path = #enter your path here
cv2.imwrite(output_path, result)

# Display the results
plt.figure(figsize=(15, 15))

plt.subplot(1, 3, 1)
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.title("Original Image")
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(final_mask, cmap='gray')
plt.title("Final Mask")
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(cv2.cvtColor(contour_image, cv2.COLOR_BGR2RGB))
plt.title("Largest Contour")
plt.axis('off')

plt.subplot(1, 3, 3)
plt.imshow(result)
plt.title("Connected Red Area with Transparency")
plt.axis('off')

plt.show()
